## Importación de Librerías

- para correr el Google Cloud
-  8 vCPU
-  16 GB memoria RAM

In [6]:
# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1815134,97.0,2933771,156.7,2933771,156.7
Vcells,3194144,24.4,8388608,64.0,5244366,40.1


In [7]:
# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1815134,97.0,2933771,156.7,2933771,156.7
Vcells,3194140,24.4,8388608,64.0,5244366,40.1


In [8]:
require("data.table")
require("lightgbm")

# Definición de Parametros

In [9]:
# defino los parametros de la corrida, en una lista, la variable global  PARAM
#  muy pronto esto se leera desde un archivo formato .yaml
PARAM <- list()
PARAM$experimento <- "KA5243"

PARAM$input$dataset <- "./competencia_02.csv.gz"

# meses donde se entrena el modelo
PARAM$input$training <- c(202101, 202102, 202103, 202104, 202105)
PARAM$input$future <- c(202107) # meses donde se aplica el modelo

PARAM$finalmodel$semilla <- 279523

PARAM$finalmodel$num_iterations <- 101
PARAM$finalmodel$learning_rate <- 0.164694054487155
PARAM$finalmodel$feature_fraction <- 0.765782067373278
PARAM$finalmodel$min_data_in_leaf <- 16
PARAM$finalmodel$num_leaves <- 923


PARAM$finalmodel$max_bin <- 31

# Aquí empieza el programa

In [10]:
# cargo el dataset donde voy a entrenar
dataset <- fread(PARAM$input$dataset, stringsAsFactors = TRUE)

In [11]:
#setwd(".Soluciones kaggle")

ERROR: Error in setwd("./Soluciones kaggle"): cannot change working directory


In [12]:
# paso la clase a binaria que tome valores {0,1}  enteros
# set trabaja con la clase  POS = { BAJA+1, BAJA+2 }
# esta estrategia es MUY importante
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2", "BAJA+1"), 1L, 0L)]

In [13]:
# los campos que se van a utilizar
campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [14]:
# establezco donde entreno
dataset[, train := 0L]
dataset[foto_mes %in% PARAM$input$training, train := 1L]

In [15]:
#--------------------------------------
# creo las carpetas donde van los resultados
# creo la carpeta donde va el experimento
dir.create("./exp/", showWarnings = FALSE)
dir.create(paste0("./exp/", PARAM$experimento, "/"), showWarnings = FALSE)

# Establezco el Working Directory DEL EXPERIMENTO
setwd(paste0("./exp/", PARAM$experimento, "/"))

In [16]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[train == 1L, campos_buenos, with = FALSE]),
  label = dataset[train == 1L, clase01]
)

In [17]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
modelo <- lgb.train(
  data = dtrain,
  param = list(
    objective = "binary",
    max_bin = PARAM$finalmodel$max_bin,
    learning_rate = PARAM$finalmodel$learning_rate,
    num_iterations = PARAM$finalmodel$num_iterations,
    num_leaves = PARAM$finalmodel$num_leaves,
    min_data_in_leaf = PARAM$finalmodel$min_data_in_leaf,
    feature_fraction = PARAM$finalmodel$feature_fraction,
    seed = PARAM$finalmodel$semilla
  )
)

[LightGBM] [Info] Number of positive: 9507, number of negative: 805631
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.203680 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3680
[LightGBM] [Info] Number of data points in the train set: 815138, number of used features: 154
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011663 -> initscore=-4.439597
[LightGBM] [Info] Start training from score -4.439597


In [18]:
#--------------------------------------
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)

In [19]:
# aplico el modelo a los datos sin clase
dapply <- dataset[foto_mes == PARAM$input$future]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dapply[, campos_buenos, with = FALSE])
)

# genero la tabla de entrega
tb_entrega <- dapply[, list(numero_de_cliente, foto_mes)]
tb_entrega[, prob := prediccion]

# grabo las probabilidad del modelo
fwrite(tb_entrega,
  file = "prediccion.txt",
  sep = "\t"
)

In [20]:
# ordeno por probabilidad descendente
setorder(tb_entrega, -prob)

In [21]:
# genero archivos con los  "envios" mejores
# deben subirse "inteligentemente" a Kaggle para no malgastar submits
# si la palabra inteligentemente no le significa nada aun
# suba TODOS los archivos a Kaggle
# espera a la siguiente clase sincronica en donde el tema sera explicado

cortes <- seq(8000, 13000, by = 500)
for (envios in cortes) {
  tb_entrega[, Predicted := 0L]
  tb_entrega[1:envios, Predicted := 1L]

  fwrite(tb_entrega[, list(numero_de_cliente, Predicted)],
    file = paste0(PARAM$experimento, "_", envios, ".csv"),
    sep = ","
  )
}

cat("\n\nLa generacion de los archivos para Kaggle ha terminado\n")



La generacion de los archivos para Kaggle ha terminado
